In [1]:
library(tidyverse)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.4.0      v purrr   0.3.4 
v tibble  3.1.6      v dplyr   1.0.10
v tidyr   1.2.0      v stringr 1.4.1 
v readr   2.1.2      v forcats 0.5.1 
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
sales_Data <- read.csv('competitive-data-science-predict-future-sales/sales_train.csv')
testData <- read.csv('competitive-data-science-predict-future-sales/test.csv')
itemsData <- read.csv('competitive-data-science-predict-future-sales/items.csv')
shopsData <- read.csv('competitive-data-science-predict-future-sales/shops.csv')
itemsCatData <- read.csv('competitive-data-science-predict-future-sales/item_categories.csv')

In [3]:
sales_Data

date,date_block_num,shop_id,item_id,item_price,item_cnt_day
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
02.01.2013,0,59,22154,999.00,1
03.01.2013,0,25,2552,899.00,1
05.01.2013,0,25,2552,899.00,-1
06.01.2013,0,25,2554,1709.05,1
15.01.2013,0,25,2555,1099.00,1
10.01.2013,0,25,2564,349.00,1
02.01.2013,0,25,2565,549.00,1
04.01.2013,0,25,2572,239.00,1
11.01.2013,0,25,2572,299.00,1


In [4]:
sales_Data$date<-as.Date(sales_Data$date,format = "%d.%m.%Y")
sales_Data<-sales_Data %>% arrange(sales_Data$date)

In [5]:
train_monthly<-sales_Data%>%inner_join(itemsData,by = "item_id")%>%filter((item_price<5000)&(item_cnt_day < 1000)&(item_price > 0) & (item_cnt_day >= 0))%>%transmute(date,date_block_num,shop_id,item_category_id, item_id, item_price, item_cnt_day,revenue=item_price*item_cnt_day)
train_monthly[train_monthly['shop_id'] == 0, 'shop_id'] = 57
train_monthly[train_monthly['shop_id']== 1, 'shop_id'] = 58
train_monthly[train_monthly['shop_id'] == 11, 'shop_id'] = 10
train_monthly[train_monthly['shop_id'] == 40, 'shop_id'] = 39
train_monthly

date,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day,revenue
<date>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>
2013-01-01,0,19,40,18976,399,1,399
2013-01-01,0,19,55,18284,199,1,199
2013-01-01,0,19,55,18320,199,1,199
2013-01-01,0,19,55,18329,299,1,299
2013-01-01,0,19,62,19367,399,1,399
2013-01-01,0,19,40,18823,299,1,299
2013-01-01,0,19,30,19413,399,1,399
2013-01-01,0,19,62,19366,399,1,399
2013-01-01,0,19,40,18721,149,1,149


In [6]:
train_monthly<-train_monthly%>%transmute(date_block_num,shop_id, item_category_id, item_id,item_cnt_day,item_price,revenue)%>%group_by(date_block_num,shop_id, item_category_id, item_id)%>%summarize(item_cnt_month=sum(item_cnt_day),mean_item_cnt=mean(item_cnt_day),mean_item_price=mean(item_price),revenue_month=sum(revenue))
train_monthly

`summarise()` has grouped output by 'date_block_num', 'shop_id', 'item_category_id'. You can override using the
`.groups` argument.


date_block_num,shop_id,item_category_id,item_id,item_cnt_month,mean_item_cnt,mean_item_price,revenue_month
<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
0,2,2,5572,9,1.285714,1532.857,13910.00
0,2,2,5573,2,1.000000,924.000,1848.00
0,2,2,5575,4,1.000000,955.005,3820.02
0,2,2,5576,3,1.000000,2490.000,7470.00
0,2,2,5632,1,1.000000,2390.000,2390.00
0,2,2,5633,1,1.000000,2298.000,2298.00
0,2,2,5643,1,1.000000,2390.000,2390.00
0,2,2,5649,1,1.000000,2190.000,2190.00
0,2,2,13510,2,1.000000,3790.000,7580.00


In [7]:
testData["date_block_num"] <- 34
testData

ID,shop_id,item_id,date_block_num
<int>,<int>,<int>,<dbl>
0,5,5037,34
1,5,5320,34
2,5,5233,34
3,5,5232,34
4,5,5268,34
5,5,5039,34
6,5,5041,34
7,5,5046,34
8,5,5319,34


In [8]:
shop_ids<-unique(testData[,'shop_id'])
item_ids<-unique(testData[,'item_id'])
months<-0:33
empty_df<-expand.grid(date_block_num=months,shop_id=shop_ids,item_id=item_ids)
empty_df<-rbind(empty_df,testData[,c('date_block_num','shop_id','item_id')])
empty_df

date_block_num,shop_id,item_id
<dbl>,<int>,<int>
0,5,5037
1,5,5037
2,5,5037
3,5,5037
4,5,5037
5,5,5037
6,5,5037
7,5,5037
8,5,5037


In [9]:
train_monthly<-empty_df%>%left_join(train_monthly,by=c('date_block_num','shop_id','item_id'))
train_monthly[is.na(train_monthly)]<-0
train_monthly<-train_monthly[,c('date_block_num','shop_id','item_id','item_cnt_month','mean_item_cnt','mean_item_price','revenue_month')]%>%inner_join(itemsData,by = "item_id")
train_monthly<-train_monthly[,c('date_block_num','shop_id','item_category_id','item_id','item_cnt_month','mean_item_cnt','mean_item_price','revenue_month')]
train_monthly['item_cnt_month']<-lapply(train_monthly['item_cnt_month'],function(x){if (x>20) 20 else(if(x<0) 0 else x)})
train_monthly['month']<-train_monthly['date_block_num']%%12
train_monthly

Warning message in if (x > 20) 20 else (if (x < 0) 0 else x):
"the condition has length > 1 and only the first element will be used"
Warning message in if (x < 0) 0 else x:
"the condition has length > 1 and only the first element will be used"


date_block_num,shop_id,item_category_id,item_id,item_cnt_month,mean_item_cnt,mean_item_price,revenue_month,month
<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,5,19,5037,0,0,0,0,0
1,5,19,5037,0,0,0,0,1
2,5,19,5037,0,0,0,0,2
3,5,19,5037,0,0,0,0,3
4,5,19,5037,0,0,0,0,4
5,5,19,5037,0,0,0,0,5
6,5,19,5037,0,0,0,0,6
7,5,19,5037,0,0,0,0,7
8,5,19,5037,0,0,0,0,8


In [10]:
temp<- train_monthly[,c("date_block_num", "shop_id","item_id","item_cnt_month","revenue_month","mean_item_cnt","mean_item_price")]
for (i in 1:3){
    shifted<-temp
    names(shifted) <-c("date_block_num", "shop_id", "item_id", sprintf("item_cnt_month_lag_%d",i),sprintf("revenue_month_lag_%d",i),sprintf("mean_item_cnt_lag_%d",i),sprintf("mean_item_price_lag_%d",i))
    shifted[,"date_block_num"]<-shifted[,"date_block_num"]+i
    train_monthly<-train_monthly%>%left_join(shifted,by=c('date_block_num','shop_id','item_id'))
}
train_monthly[is.na(train_monthly)]<-0

In [11]:
minMax <- function(x) {
  (x - min(x)) / (max(x) - min(x))
}
y_min<-min(train_monthly$item_cnt_month)
y_max<-max(train_monthly$item_cnt_month)
train_monthly[,c('item_cnt_month','mean_item_cnt','mean_item_price','revenue_month','month','item_cnt_month_lag_1','revenue_month_lag_1','mean_item_cnt_lag_1','mean_item_price_lag_1','item_cnt_month_lag_2','revenue_month_lag_2','mean_item_cnt_lag_2','mean_item_price_lag_2','item_cnt_month_lag_3','revenue_month_lag_3','mean_item_cnt_lag_3','mean_item_price_lag_3')]<-apply(train_monthly[,c('item_cnt_month','mean_item_cnt','mean_item_price','revenue_month','month','item_cnt_month_lag_1','revenue_month_lag_1','mean_item_cnt_lag_1','mean_item_price_lag_1','item_cnt_month_lag_2','revenue_month_lag_2','mean_item_cnt_lag_2','mean_item_price_lag_2','item_cnt_month_lag_3','revenue_month_lag_3','mean_item_cnt_lag_3','mean_item_price_lag_3')],2,minMax)

In [12]:
train<-train_monthly[train_monthly$date_block_num<33,-which(names(train_monthly) %in% c('mean_item_price','revenue_month','mean_item_cnt'))]
valid<-train_monthly[train_monthly$date_block_num==33,-which(names(train_monthly) %in% c('mean_item_price','revenue_month','mean_item_cnt'))]
test<-train_monthly[train_monthly$date_block_num==33,-which(names(train_monthly) %in% c('item_cnt_month','mean_item_price','revenue_month','mean_item_cnt'))]
tem<-lm(item_cnt_month~.,data=train)
tem


Call:
lm(formula = item_cnt_month ~ ., data = train)

Coefficients:
          (Intercept)         date_block_num                shop_id  
            8.240e-05              2.002e-07              3.118e-08  
     item_category_id                item_id                  month  
           -1.196e-06             -1.214e-09              8.211e-05  
 item_cnt_month_lag_1    revenue_month_lag_1    mean_item_cnt_lag_1  
            7.171e-01             -1.907e-01             -2.017e-01  
mean_item_price_lag_1   item_cnt_month_lag_2    revenue_month_lag_2  
            1.374e-03              5.766e-02              7.766e-03  
  mean_item_cnt_lag_2  mean_item_price_lag_2   item_cnt_month_lag_3  
            1.765e-02              2.952e-04              1.607e-01  
  revenue_month_lag_3    mean_item_cnt_lag_3  mean_item_price_lag_3  
           -4.288e-02             -3.411e-02              5.066e-04  


In [13]:
mean((predict(tem, train)-train[,'item_cnt_month'])^2)

[1] 2.246538e-06

In [14]:
summary(tem)


Call:
lm(formula = item_cnt_month ~ ., data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.51803 -0.00009 -0.00006 -0.00003  0.62292 

Coefficients:
                        Estimate Std. Error  t value Pr(>|t|)    
(Intercept)            8.240e-05  2.372e-06   34.741  < 2e-16 ***
date_block_num         2.002e-07  6.083e-08    3.291 0.000997 ***
shop_id                3.118e-08  3.210e-08    0.971 0.331396    
item_category_id      -1.196e-06  3.522e-08  -33.945  < 2e-16 ***
item_id               -1.214e-09  9.379e-11  -12.946  < 2e-16 ***
month                  8.211e-05  1.906e-06   43.071  < 2e-16 ***
item_cnt_month_lag_1   7.171e-01  5.305e-04 1351.876  < 2e-16 ***
revenue_month_lag_1   -1.907e-01  4.107e-04 -464.242  < 2e-16 ***
mean_item_cnt_lag_1   -2.017e-01  9.781e-04 -206.180  < 2e-16 ***
mean_item_price_lag_1  1.374e-03  1.298e-05  105.863  < 2e-16 ***
item_cnt_month_lag_2   5.766e-02  6.483e-04   88.934  < 2e-16 ***
revenue_month_lag_2    7.766e-03  4

In [15]:
mean((predict(tem, valid)-valid[,'item_cnt_month'])^2)

[1] 1.807155e-06

In [16]:
test_y<-(predict(tem, test)*(y_max-y_min))+y_min
test_y<-round(test_y)
test_y

34      68     102     136     170     204     238     272     306     340 
      1       1       1       0       1       2       0       0       5      12 
    374     408     442     476     510     544     578     612     646     680 
      0       3       1       4       1       1       2       2       4       1 
    714     748     782     816     850     884     918     952     986    1020 
      4       3       1       3       3       1       1       0       0       3 
   1054    1088    1122    1156    1190    1224    1258    1292    1326    1360 
      0       1       0       3       3       5       3       1       1       0 
   1394    1428    1462    1496    1530    1564    1598    1632    1666    1700 
      0       1       0       0       0       0       0       0       0       0 
   1734    1768    1802    1836    1870    1904    1938    1972    2006    2040 
      0       0       0       0       0       0       0       0       0       0 
   2074    2108    2142    2176    2210    2244    2278    2312    2346    2380 
      0       0       0       0       0       0       0       0       0       0 
   2414    2448    2482    2516    2550    2584    2618    2652    2686    2720 
      0       0       0       0       0       0       0       0       0       0 
   2754    2788    2822    2856    2890    2924    2958    2992    3026    3060 
      0       0       0       0       2       1       2       2       3       1 
   3094    3128    3162    3196    3230    3264    3298    3332    3366    3400 
      0       1       4       4       1       3       1       1       1       3 
   3434    3468    3502    3536    3570    3604    3638    3672    3706    3740 
      1       2       0       1       1       1       2       1       0       3 
   3774    3808    3842    3876    3910    3944    3978    4012    4046    4080 
      2       3       1       2       0       1       0       1       2       4 
   4114    4148    4182    4216    4250    4284    4318    4352    4386    4420 
      0       0       1       1       1       1       0       0       0       0 
   4454    4488    4522    4556    4590    4624    4658    4692    4726    4760 
      1       1       0       0       2       1       0       2       0       1 
   4794    4828    4862    4896    4930    4964    4998    5032    5066    5100 
      1       0       1       0       0       1       3       0       0       1 
   5134    5168    5202    5236    5270    5304    5338    5372    5406    5440 
      1       1       1       1       1       1       0       2       0       1 
   5474    5508    5542    5576    5610    5644    5678    5712    5746    5780 
      3       2       1       1       1       0       1       1       0       0 
   5814    5848    5882    5916    5950    5984    6018    6052    6086    6120 
      0       0       0       0       0       0       0       0       0       0 
   6154    6188    6222    6256    6290    6324    6358    6392    6426    6460 
      0       0       0       0       0       0       0       0       0       0 
   6494    6528    6562    6596    6630    6664    6698    6732    6766    6800 
      0       0       0       0       0       0       0       0       0       0 
   6834    6868    6902    6936    6970    7004    7038    7072    7106    7140 
      0       0       0       0       0       0       0       0       0       0 
   7174    7208    7242    7276    7310    7344    7378    7412    7446    7480 
      1       1       1       1       1       2       1       1       3       3 
   7514    7548    7582    7616    7650    7684    7718    7752    7786    7820 
      0       2       1       2       1       1       2       1       1       1 
   7854    7888    7922    7956    7990    8024    8058    8092    8126    8160 
      6       1       4       3       3       1       2       1       1       3 
   8194    8228    8262    8296    8330    8364    8398    8432    8466    8500 
      0       1       0       2     

In [17]:
print(length(test_y))

[1] 214200


In [21]:
results <- data.frame (ID  = testData$ID,
                  item_cnt_month = test_y
                  )
results

,ID,item_cnt_month
,<int>,<dbl>
34,0,1
68,1,1
102,2,1
136,3,0
170,4,1
204,5,2
238,6,0
272,7,0
306,8,5


In [22]:
write.csv(results,file="results/data.csv",row.names = FALSE)